In [131]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline

#### Метрика:

In [570]:
def SMAPE(y_pred, y_true):
    return 200 * (np.sum( np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))) / y_pred.size

### Считывание данных

In [482]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

#### Посмотрим на данные

In [483]:
train.head()

,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,0,123438,2012,52,1,20442076,4915.0,38056.0,40185.0,45733.0,...,39423.0,41765.0,52590.0,31452.0,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0
1,1,58410,2012,52,1,20441997,2230.0,18817.0,20110.0,26368.0,...,22830.0,25230.0,27850.0,21390.0,27090.0,23170.0,29705.0,19140.0,22055.0,23200.0
2,2,163930,2012,52,1,20441990,5695.0,47480.0,47619.0,89708.0,...,14930.0,44290.0,46412.0,29320.0,21140.0,28406.0,65056.0,31886.0,48750.0,36520.0
3,3,53902,2012,52,1,20441989,1995.0,17146.0,20066.0,27070.0,...,15120.0,12480.0,19780.0,7990.0,8230.0,10650.0,21920.0,13040.0,9780.0,9630.0
4,4,105970,2012,52,1,20441988,6515.0,49262.0,50045.0,95167.0,...,18872.0,19328.0,37168.0,13570.0,19760.0,20208.0,34745.0,18442.0,24700.0,21793.0


In [484]:
train.columns

Index(['Num', 'y', 'year', 'week', 'shift', 'item_id', 'f1', 'f2', 'f3', 'f4',
       'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15',
       'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25',
       'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35',
       'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45',
       'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55',
       'f56', 'f57', 'f58', 'f59', 'f60'],
      dtype='object')

In [485]:
test.head()

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
1,348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
2,348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
3,348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
4,348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


In [486]:
test.columns

Index(['Num', 'year', 'week', 'shift', 'item_id', 'f1', 'f2', 'f3', 'f4', 'f5',
       'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16',
       'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26',
       'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36',
       'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46',
       'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56',
       'f57', 'f58', 'f59', 'f60'],
      dtype='object')

### нужно определить y - продажи за данную неделю

# Работа с данными

Посмотрим на какую-нибудь строчку полностью:

In [487]:
print((train.loc[0]))

Num               0.0
y            123438.0
year           2012.0
week             52.0
shift             1.0
item_id    20442076.0
f1             4915.0
f2            38056.0
f3            40185.0
f4            45733.0
f5            59710.0
f6            39982.0
f7            45846.0
f8            43680.0
f9            48325.0
f10           42685.0
f11           40605.0
f12           44601.0
f13           41965.0
f14           56221.0
f15           34260.0
f16           39914.0
f17           42322.0
f18           48903.0
f19           42090.0
f20           36690.0
f21           39423.0
f22           41765.0
f23           52590.0
f24           31452.0
              ...    
f31            4915.0
f32           38056.0
f33           40185.0
f34           45733.0
f35           59710.0
f36           39982.0
f37           45846.0
f38           43680.0
f39           48325.0
f40           42685.0
f41           40605.0
f42           44601.0
f43           41965.0
f44           56221.0
f45       

Видим, что некоторые признаки дублируются (f1 - f31, f2 - f32, ..)

Проверим, так ли для всех данных:

In [488]:
first_par_of_columns = train.columns[6:36]
second_par_of_columns = train.columns[36:]
np.all([np.all(train[first_par_of_columns[i]] == train[second_par_of_columns[i]]) for i in range(30)])

True

Да, действительно, все признаки 31-60 совпадают. Тогда просто удалим их.

In [489]:
train.drop(second_par_of_columns, axis=1, inplace=True)

То же самое для test:

In [490]:
first_par_of_columns = test.columns[5:35]
second_par_of_columns = test.columns[35:]
np.all([np.all(test[first_par_of_columns[i]] == test[second_par_of_columns[i]]) for i in range(30)])

True

In [491]:
test.drop(second_par_of_columns, axis=1, inplace=True)

#### Теперь посмотрим на какой-нибудь один товар:

In [492]:
train[train['item_id'] == 20442092]

,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
15,15,1175,2012,52,1,20442092,10.0,270.0,980.0,6140.0,...,180.0,370.0,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0
235,235,241,2013,1,2,20442092,10.0,270.0,980.0,6140.0,...,180.0,370.0,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0
478,478,161,2013,2,3,20442092,10.0,270.0,980.0,6140.0,...,180.0,370.0,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0
696,3468,241,2013,1,1,20442092,270.0,980.0,6140.0,8292.0,...,370.0,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0
939,3711,161,2013,2,2,20442092,270.0,980.0,6140.0,8292.0,...,370.0,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0
1157,3929,370,2013,3,3,20442092,270.0,980.0,6140.0,8292.0,...,370.0,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0
1403,6946,161,2013,2,1,20442092,980.0,6140.0,8292.0,780.0,...,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0,150.0
1621,7164,370,2013,3,2,20442092,980.0,6140.0,8292.0,780.0,...,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0,150.0
1868,7411,273,2013,4,3,20442092,980.0,6140.0,8292.0,780.0,...,151.0,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0,150.0
2081,10393,370,2013,3,1,20442092,6140.0,8292.0,780.0,426.0,...,220.0,220.0,450.0,290.0,240.0,210.0,830.0,730.0,150.0,100.0


Видим, что в некоторых строчках одна и та же дана, но разное смещение признаков. Или наоборот, признаки совпадают, а даты нет, из-за смещения признаков.

### Сделаем сдвиг по shift

Для удобства переведем (year, week ) в cum_week - номер недели.

In [493]:
from copy import copy

In [494]:
train['cum_week'] = (train['year'] - 2012) * 52 + train['week']
train.loc[train['year'] == 2015, 'cum_week'] += 1

In [495]:
train['features_week'] = train['cum_week'] - train['shift']
train.loc[train['year'] == 2015, 'features_week'] += 1

feature_week y -> y с cum_sum == frature_weekm

In [502]:
for item in np.unique(train['item_id']):
    item_table = train[train['item_id'] == item]
    for id_ in item_table.index:
        fw = train.loc[id_, 'features_week']        
        new_y = np.array(item_table.loc[item_table['cum_week'] == fw, 'y'])
        if new_y.size != 0:
            train.loc[id_, 'y'] = new_y[0]
            train.loc[id_, 'shift'] = 0           

In [508]:
train.drop(train.loc[train['shift']!=0].index, inplace=True)

In [509]:
model = LinearRegression()
model.fit(train[train.columns[-32:-2]], train['y'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [510]:
model.coef_

array([ 0.01006577, -0.00974424, -0.01720868,  0.03466723, -0.03597839,
        0.00633532,  0.01782816, -0.00513784, -0.00288026,  0.00292973,
       -0.01142498,  0.00689255,  0.0117644 ,  0.01088067, -0.06456222,
        0.01630652, -0.00371452, -0.00554449,  0.01416301,  0.02535637,
       -0.04455114,  0.0495016 ,  0.00179287,  0.0054044 ,  0.02368869,
        0.01413663,  0.0032382 ,  0.03938407, -0.0237843 ,  1.5337828 ])

## Кросс-валидация

In [565]:
train = train.sort_values('cum_week')

In [566]:
coef = (train.shape[0] - test.shape[0]) / train.shape[0]

In [567]:
X = train[:int(train.shape[0] * coef)].drop(['y'], axis=1)[train.columns[-32:-2]]
X_test = train[int(train.shape[0] * coef):].drop(['y'], axis=1)[train.columns[-32:-2]]

y = train[:int(train.shape[0] * coef)]['y']
y_test = train[int(train.shape[0] * coef):]['y']

In [568]:
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X_test)

In [571]:
SMAPE(y_test, y_pred)

48.560004551255105

In [548]:
model.coef_

array([  2.46713588e-09,   1.75244106e-08,   2.12912007e-08,
        -2.42397659e-08,   3.75659209e-08,   2.70763498e-08,
        -2.98570176e-08,   2.74789802e-08,  -3.01412320e-08,
         1.78406964e-08,  -2.65359771e-08,   2.50311773e-08,
        -4.89192570e-08,  -1.70239086e-08,  -1.92949780e-08,
        -1.33481429e-08,  -2.81998247e-09,   1.20833005e-09,
         1.87848523e-08,   2.04077654e-08,   1.26256743e-09,
         3.26828629e-09,  -4.59214512e-08,   9.23986304e-09,
        -4.38042572e-09,   2.34416636e-09,   1.91254340e-08,
        -2.45051148e-08,   1.07589285e-08,   1.60999999e+00])

In [572]:
m_gbr = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
m_gbr.fit(X, y)
gbr_pred = m_gbr.predict(X_test)
SMAPE(gbr_pred, y_test)

49.8142399245073

У меня не получилось сделать так, чтобы он соответствовал leaderboard.

# Другие идеи:

Пробовала смешивать модели.
Пробовала предсказать f30 - чтобы использовать эти предсказания, при применении линейной модели в 2015ом году. 
Но ни деревья на всех данных, ни ленейная регрессия от признаков давала плохой.

### Другая интересная информация о данных:

In [48]:
Train_Unique_ID_Items = np.unique(train['item_id'])

In [49]:
Test_Unique_ID_Items = np.unique(test['item_id'])

In [50]:
Train_Unique_ID_Items.size

286

In [51]:
Test_Unique_ID_Items.size

227